Zde hledáme Wikidata IDs toho autorstva, které je nemá vyplněné v databázi autorit.

In [2]:
import gc
import os
import re
import json
import requests
import pandas as pd

In [3]:
kam = 'data_raw/wikidata_doplneni'
if not os.path.exists(kam):
    os.makedirs(kam)

In [4]:
vsechny_jsony = [f for f in os.listdir(kam)]

In [5]:
vyhledane = set([x.split("_")[0] for x in vsechny_jsony])

In [6]:
df = pd.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))

In [7]:
df = df.explode('100_7')

In [8]:
df['100_7'].nunique()

171303

In [9]:
df.groupby('100_7').size().nlargest(1000)

100_7
jx20060515016    2960
jx20050830020    2373
jn20001103529    1652
jn19981000826    1468
jk01120043       1385
                 ... 
jn19990209875      65
jn19990210374      65
jn20000710062      65
jn20000810185      65
jn99240001194      65
Length: 1000, dtype: int64

In [10]:
opakovane = df.groupby('100_7').size()
opakovane = opakovane.sort_values(ascending=False)
opakovane = opakovane[opakovane > 1].index.to_list()
len(opakovane)

90630

In [11]:
autority = pd.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [12]:
autority

,024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,...,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader
001,,,,,,,,,,,,,,,,,,,,,
jk01010001,None,None,[1931],None,[jk01010001],"[Abraham, Jiří,]",[1931-],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 14.12.1931 v Osvračíně. MUDr., odborn...",None,nz a22 n 4500
jk01010002,None,None,[1890],[1933],[jk01010002],"[Abraham, Josef,]",[1890-1933],[1],None,None,...,None,None,None,None,None,None,None,[Narozen 19.11.1890 v Osvračíně u Horšova Týna...,None,nz a22 n 4500
jk01010003,None,None,[1957],None,[jk01010003],"[Abraham, Pavel,]",[1957-],[1],None,None,...,None,None,None,None,None,None,None,[Narozen roku 1957. Student filosofické fakult...,None,nz a22 n 4500
jk01010004,None,None,[1889],[1963],[jk01010004],"[Absolon, Josef,]",[1889-1963],[1],None,"[Odrlice, Senice na Hané, Česko]",...,None,None,None,None,None,None,None,"[Narozen 27. 1. 1889 v Odrlicích na Moravě, ze...",None,cz a22 n 4500
jk01010005,"[isni, wikidata]","[0000000109542912, Q1063371]",[1877],[1960],[jk01010005],"[Absolon, Karel,]",[1877-1960],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 16.6.1877 v Boskovicích, zemřel 6.10....",None,cz a22 n 4500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xx0323367,None,None,None,None,[xx0323367],"[Birhanu, Abigail]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a učitelka výtvarné výcho...,None,nz a22 n 4500
xx0323368,None,None,None,None,[xx0323368],"[Latimer, Khadesia]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a výtvarná pedagožka zamě...,None,nz a22 n 4500
xx0323369,None,None,None,None,[xx0323369],"[Liz, Paula]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a výtvarná pedagožka.],None,nz a22 n 4500


In [13]:
autority = autority[autority['024_2'].notnull()]
autority = autority[autority['024_a'].notnull()]

In [14]:
autority = autority.explode(['024_a','024_2'])

In [15]:
autority_s_wikidaty = set(autority[autority['024_2'] == 'wikidata']['024_a'].index.to_list())

In [16]:
chybi = [x for x in opakovane if x not in autority_s_wikidaty]

In [17]:
del df
del autority
gc.collect()

0

In [18]:
len(chybi)

82368

In [19]:
# Wikidata SPARQL endpoint
sparql_endpoint = "https://query.wikidata.org/sparql"

# Function to get Wikidata ID from NL CR AUT ID
def get_wikidata_id(nl_cr_aut_id):
    query = f"""
    SELECT ?item WHERE {{
      ?item wdt:P691 "{nl_cr_aut_id}".
    }}
    """
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(sparql_endpoint, params={"query": query}, headers=headers)
    data = response.json()

    if "results" in data and "bindings" in data["results"]:
        bindings = data["results"]["bindings"]
        if bindings:
            return bindings[0]["item"]["value"].split("/")[-1]
    return None

In [20]:
vyhledane = set(vyhledane)

In [21]:
co_stahovat = [x for x in chybi if x not in vyhledane]

In [22]:
len(co_stahovat)

0

In [23]:
pocitadlo = 0
for nl_cr_aut_id in co_stahovat:
    pocitadlo += 1
    print(f"{pocitadlo:005}: {nl_cr_aut_id}")
    slovnik = {}
    slovnik['100_7'] = nl_cr_aut_id
    wikidata_id = get_wikidata_id(nl_cr_aut_id)
    if wikidata_id:
        slovnik['024_a'] = wikidata_id
    else:
        slovnik['024_a'] = ""
    with open(os.path.join(kam, f"{nl_cr_aut_id}_{slovnik['024_a']}.json"), "w+", encoding="utf-8") as file:
        file.write(json.dumps(slovnik))